In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/PHD/SemEval2023

/content/gdrive/MyDrive/PHD/SemEval2023


In [ ]:
# import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd

In [ ]:
from collections import Counter

In [ ]:
target_column = "label_category"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

In [ ]:
texts = data["text"].values
# words = [preprocess(s) for s in texts]
labels = data[target_column].values

In [ ]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

  label_map = {
      0: '1. threats, plans to harm and incitement',
      1: '2. derogation', 
      2: '3. animosity', 
      3: '4. prejudiced discussions',
      '1. threats, plans to harm and incitement':0,
      '2. derogation':1,
      '3. animosity':2,
      '4. prejudiced discussions':3,
  }
elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

#   label_map = {
#       0: '1.1 threats of harm',
#       1: '1.2 incitement and encouragement of harm',
#       2: '2.1 descriptive attacks',
#       3: '2.2 aggressive and emotive attacks',
#       4: '2.3 dehumanising attacks & overt sexual objectification',
#       5: '3.1 casual use of gendered slurs, profanities, and insults',
#       6: '3.2 immutable gender differences and gender stereotypes',
#       7: '3.3 backhanded gendered compliments',
#       8: '3.4 condescending explanations or unwelcome advice',
#       9: '4.1 supporting mistreatment of individual women',
#       10: '4.2 supporting systemic discrimination against women as a group',
#       '1.1 threats of harm': 0,
#       '1.2 incitement and encouragement of harm': 1,
#       '2.1 descriptive attacks': 2,
#       '2.2 aggressive and emotive attacks': 3,
#       '2.3 dehumanising attacks & overt sexual objectification': 4,
#       '3.1 casual use of gendered slurs, profanities, and insults': 5,
#       '3.2 immutable gender differences and gender stereotypes': 6,
#       '3.3 backhanded gendered compliments': 7,
#       '3.4 condescending explanations or unwelcome advice': 8,
#       '4.1 supporting mistreatment of individual women': 9,
#       '4.2 supporting systemic discrimination against women as a group': 10,
#   }
# else:
#   raise Exception("Unknown label column")

In [ ]:
import torch
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(data, labels, random_state=42, test_size=0.2)

,rewire_id,text,label_sexist,label_category,label_vector
8825,sexism2022_english-19551,My girlfriend was reduced to tears when I show...,sexist,"1. threats, plans to harm and incitement",1.2 incitement and encouragement of harm
12156,sexism2022_english-10678,This whole comic confirms the blackpill God I ...,sexist,2. derogation,2.2 aggressive and emotive attacks
13271,sexism2022_english-16652,"I can, from personal experience vouch for this...",sexist,3. animosity,3.2 immutable gender differences and gender st...
12816,sexism2022_english-8208,There is no such thing as a lesbian. Its just ...,sexist,2. derogation,2.1 descriptive attacks
7116,sexism2022_english-18994,What a stupid question! NO MAN...would humilia...,sexist,2. derogation,2.1 descriptive attacks
...,...,...,...,...,...
4466,sexism2022_english-7734,I watched like couple other videos and apparen...,sexist,2. derogation,2.3 dehumanising attacks & overt sexual object...
4635,sexism2022_english-9987,The she-god mentality of women is going down t...,sexist,2. derogation,2.3 dehumanising attacks & overt sexual object...
5317,sexism2022_english-13884,Chad is rawdogging her in the all gender bathroom,sexist,2. derogation,2.3 dehumanising attacks & overt sexual object...
3502,sexism2022_english-7048,Because avoiding rape is entirely within a wom...,sexist,4. prejudiced discussions,4.1 supporting mistreatment of individual women


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()


def preprocess(sent):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  words = [lemmatizer.lemmatize(w) for w in words]
  return words

def dummy_word_tokenize(sent):
  return sent

vectorizer = CountVectorizer(
    preprocessor=preprocess,
    tokenizer=dummy_word_tokenize
)

In [ ]:
X_train = vectorizer.fit_transform(train["text"].values)
X_test = vectorizer.transform(test["text"].values)

In [ ]:
"#Vocab", len(vectorizer.get_feature_names_out())

('#Vocab', 7574)

In [ ]:
y_train = [label_map[l] for l in train_labels]
y_test = [label_map[l] for l in test_labels]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [ ]:
y_predict = lr.predict(X_test)

In [ ]:
len(y_test), len(y_predict), X_test.shape

(680, 680, (680, 7574))

In [ ]:
print("LogisticRegression Accuracy %.3f" % metrics.f1_score(y_test, y_predict, average='macro'))

LogisticRegression Accuracy 0.445
